In [1]:
import numpy as np

In [2]:
from phevaluator.evaluator import evaluate_cards

p1 = evaluate_cards("2c", "3c", "4c", "5c", "As", "8s", "9s")
p2 = evaluate_cards("2c", "3c", "4c", "5c", "7s", "8s", "9s")

# Player 2 has a stronger hand

In [ ]:
from deck.card import Card, Rank, Suit
from deck.deck import Deck

In [4]:
def sort_hand(hand: np.ndarray) -> np.ndarray:
    return np.array(sorted(hand, reverse=True, key=lambda x: (x.get_rank(), x.get_tag())))

In [5]:
def is_a_pair(hand: np.ndarray) -> bool:
    return hand[0].get_rank() == hand[1].get_rank()

def compare_starting_hands(hand_one: np.ndarray, hand_two: np.ndarray) -> int:
    """
    Return 1 if hand one is stronger than hand two, -1 if hand two stronger, 0 if equal
    """
    is_hand_one_a_pair = is_a_pair(hand_one)
    is_hand_two_a_pair = is_a_pair(hand_two)
    if is_hand_one_a_pair and not is_hand_two_a_pair:
        return 1
    elif not is_hand_one_a_pair and is_hand_two_a_pair:
        return -1
    elif is_hand_one_a_pair and is_hand_two_a_pair:
        rank_one = hand_one[0].get_rank()
        rank_two = hand_two[0].get_rank()
        if rank_one == rank_two:
            return 0
        if rank_one > rank_two:
            return 1
        else:
            return -1
    else:
        hand_one = sort_hand(hand_one)
        hand_two = sort_hand(hand_two)
        if hand_one[0].get_rank() > hand_two[0].get_rank():
            return 1
        elif hand_one[0].get_rank() < hand_two[0].get_rank():
            return -1
        else:
            if hand_one[1].get_rank() > hand_two[1].get_rank():
                return 1
            elif hand_one[1].get_rank() < hand_two[1].get_rank():
                return -1
            else:
                return 0

In [ ]:
from typing import Literal

In [ ]:
def compute_initial_effective_hand_strengths(iterations: int, suited=False):
    if suited:
        shift = 1
        name = "s"
    else:
        shift = 0
        name = "o"
    effective_hand_strengths = {}
    for i, rank_one in enumerate(list(Rank)[::-1]):
        for rank_two in list(Rank)[::-1][i+shift:]:
            deck = Deck()
            cards = np.concatenate((deck.draw_card_by_tag(str(rank_one.value) + "c"), deck.draw_card_by_tag(str(rank_two.value) + "s")))
            wins = 0
            positive_potential = 0
            negative_potential = 0
            for k in range(iterations):
                deck.shuffle()
                opponent_cards = deck.draw_cards(2)
                initial_advantage = compare_starting_hands(sort_hand(cards), sort_hand(opponent_cards))
                flop = deck.draw_cards(3)
                turn = deck.draw_cards(1)
                river = deck.draw_cards(1)
                my_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((cards, flop, turn, river))])
                opponent_strength = evaluate_cards(*[card.get_tag() for card in np.concatenate((opponent_cards, flop, turn, river))])
                if my_strength < opponent_strength:
                    wins += 1
                    if initial_advantage == -1:
                        positive_potential += 1 #won the hand with initial disadvantage
                    elif initial_advantage == 0:
                        positive_potential += 0.5
                elif my_strength == opponent_strength: # draw
                    wins += 0.5
                    if initial_advantage == -1:
                        positive_potential += 0.5
                    elif initial_advantage == 1:
                        negative_potential += 0.5
                else: 
                    if initial_advantage == 1: #lost the hand with initial advantage
                        negative_potential += 1
                    elif initial_advantage == 0:
                        negative_potential += 0.5
                deck.return_cards(opponent_cards)
                deck.return_cards(flop)
                deck.return_cards(turn)
                deck.return_cards(river)
            effective_hand_strengths[str(rank_one.value) + str(rank_two.value) + name] = (wins * (iterations - negative_potential) + (iterations - wins) * positive_potential) / iterations ** 2
    return effective_hand_strengths

In [ ]:
iterations = 100_000
suited_effective_hand_strength = compute_initial_effective_hand_strengths(iterations, True)
offsuit_effective_hand_strength = compute_initial_effective_hand_strengths(iterations, False)
effective_hand_strength = {
    **suited_effective_hand_strength,
    **offsuit_effective_hand_strength
}
for k,v in sorted(effective_hand_strength.items(), key=lambda item: item[1], reverse=True):
    print(k, v)

54s 0.59429288
76s 0.5869648625
65s 0.58473224
87s 0.58153134
98s 0.57397564
43s 0.5734459375
75s 0.5699562675
64s 0.56912449
T9s 0.56444276
53s 0.558874175
97s 0.55602122
86s 0.55517512
JTs 0.554551555
42s 0.54924784
63s 0.54890642
52s 0.543488385
85s 0.5415299675
74s 0.540173375
96s 0.5396661825
62s 0.538025125
84s 0.537868025
T7s 0.53473728
32s 0.5345065825
J9s 0.5320581075
T8s 0.52925245
73s 0.52690542
95s 0.5252623875
QJs 0.5160946625
QTs 0.5158083625
T6s 0.5117291475
J8s 0.508762635
83s 0.50711048
93s 0.50646236
72s 0.50352874
92s 0.50300168
Q9s 0.500136955
T5s 0.49540056
T4s 0.4919412925
T3s 0.4916535375
82s 0.49154918
KQs 0.488600025
94s 0.4877767625
KJs 0.4868213
J7s 0.4857401975
Q8s 0.48190545
T2s 0.48130639
J6s 0.479751915
J4s 0.474139195
J3s 0.47288381
KTs 0.47213765
J5s 0.468443095
K9s 0.4611889475
Q6s 0.4588578025
AKs 0.45763609
Q7s 0.456153
J2s 0.45337696
AQs 0.45290796
Q2s 0.45097675
Q4s 0.450058405
Q5s 0.446907275
K8s 0.44420613
Q3s 0.4410076175
AJs 0.440562575
ATs 0.4

AAo 0.71829855
KKo 0.6897993675
QQo 0.64671178
JJo 0.598821635
54s 0.59429288
76o 0.5936522175
65o 0.5927923
54o 0.59069
76s 0.5869648625
65s 0.58473224
87s 0.58153134
TTo 0.578634105
98o 0.57773674
64o 0.576188605
98s 0.57397564
43s 0.5734459375
87o 0.5718046
75o 0.57088549
75s 0.5699562675
53o 0.569623465
64s 0.56912449
T9s 0.56444276
86o 0.5643648
T9o 0.56025062
43o 0.5590236825
53s 0.558874175
63o 0.55834758
97s 0.55602122
86s 0.55517512
JTs 0.554551555
74o 0.55449776
52o 0.55328256
42o 0.55049941
42s 0.54924784
63s 0.54890642
T8o 0.544353795
52s 0.543488385
97o 0.542582675
85o 0.54167392
85s 0.5415299675
JTo 0.540781525
74s 0.540173375
96s 0.5396661825
62s 0.538025125
99o 0.53788092
84s 0.537868025
96o 0.53579808
T7s 0.53473728
32s 0.5345065825
J9o 0.5331483
J9s 0.5320581075
32o 0.529733255
T8s 0.52925245
95o 0.52764318
73s 0.52690542
62o 0.5265207775
73o 0.5264836025
84o 0.52643908
95s 0.5252623875
QJo 0.521274625
T7o 0.51985502
QTo 0.516140785
QJs 0.5160946625
QTs 0.5158083625
T

In [32]:
import json

with open("ehs.json", 'w') as f:
    json.dump(effective_hand_strengths, f)

In [ ]:
with open("waverage_strength.json", 'w') as f:
    json.dump(cards_strength, f)

In [36]:
with open("waverage_strength.json", 'r') as f:
    cards_strength = json.load(f)

In [37]:
max_std = max([x[1] for x in cards_strength.values()])
min_std = min([x[1] for x in cards_strength.values()])

In [38]:
print(max_std, min_std)

1912.0 984.0


In [34]:
for k,v in cards_strength.items():
    cards_strength[k] = (v[0], round((v[1] - min_std) / (max_std - min_std), 3))

In [35]:
for k,v in sorted(cards_strength.items(), key=lambda item: item[1][0]):
    print(k,v)

AAo (2457.0, 0.0)
KKo (2599.0, 0.062)
QQo (2713.0, 0.142)
JJo (2825.0, 0.214)
TTo (2930.0, 0.297)
99o (3033.0, 0.377)
88o (3130.0, 0.458)
77o (3234.0, 0.545)
66o (3319.0, 0.627)
55o (3411.0, 0.718)
44o (3516.0, 0.811)
33o (3622.0, 0.903)
AKs (3671.0, 0.852)
AQs (3713.0, 0.856)
22o (3723.0, 0.998)
AJs (3737.0, 0.855)
ATs (3756.0, 0.859)
KQs (3764.0, 0.88)
JTs (3789.0, 0.905)
KJs (3795.0, 0.881)
QJs (3795.0, 0.897)
KTs (3800.0, 0.891)
QTs (3807.0, 0.903)
A9s (3830.0, 0.853)
A8s (3839.0, 0.852)
T9s (3846.0, 0.905)
AKo (3862.0, 0.721)
A7s (3864.0, 0.865)
A5s (3868.0, 0.897)
J9s (3869.0, 0.895)
Q9s (3881.0, 0.886)
K9s (3886.0, 0.876)
98s (3896.0, 0.904)
A6s (3903.0, 0.873)
AQo (3906.0, 0.726)
A4s (3907.0, 0.913)
T8s (3911.0, 0.899)
AJo (3933.0, 0.737)
J8s (3933.0, 0.891)
A3s (3938.0, 0.92)
87s (3943.0, 0.915)
ATo (3948.0, 0.736)
K7s (3950.0, 0.878)
KQo (3953.0, 0.786)
K8s (3953.0, 0.874)
97s (3954.0, 0.906)
Q8s (3956.0, 0.887)
KJo (3960.0, 0.789)
QJo (3961.0, 0.824)
A2s (3966.0, 0.95)
JTo (

In [ ]:
def sort_hand(hand: np.ndarray) -> np.ndarray:
    return np.array(sorted(hand, key=lambda x: )) 

In [56]:
deck = Deck()
deck.shuffle()
cards = np.concatenate((deck.draw_card_by_tag("Qc"), deck.draw_card_by_tag("Qs"), deck.draw_card_by_tag("Qd"), deck.draw_card_by_tag("Qh")))

In [57]:
sorted(cards, reverse=True, key=lambda x: (x, x.get_tag()))

[Queen of Clubs, Queen of Spades, Queen of Diamonds, Queen of Hearts]